## Webscraping and cleaning Haspa marathon data (as of 25.07.2023)
- Source: https://haspa-marathon-hamburg.de/ergebnisse/
- Results of 2015, 2016 and 2017 can be downloaded from website. Thus simply merged them to one .csv
- Web scraped results for 2018, 2019, 2021, 2022 and 2023. No marathon was held in 2020 due to covid
- 2015 - 2017 data also has split wise times while 2018-2023 does not

In [2]:
#import required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [78]:
#repeated below for each year - 2018, 2019, 2021, 2022 and 2023
#name_url_list can be further used to build urls to each user profile
#for example - 2018
url_list = []
haspa_marathon_2018 = pd.DataFrame()


for i in range(1,17):
    url = 'https://hamburg.r.mikatiming.com/2018/?page={}&event=HML&lang=DE&num_results=500&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25'.format(i)
    url_list.append(url)

for url in url_list:
    #url =  'https://hamburg.r.mikatiming.com/2023/?page=2&event=HML&lang=DE&num_results=500&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25'
    html = requests.get(url)
    soup = BeautifulSoup(html.text,'html')

    platz = soup.find_all('div',class_='list-field type-place place-primary numeric')
    platz_list = [i.text.strip() for i in platz]

    plak = soup.find_all('div',class_= ['list-field type-place place-secondary hidden-xs numeric','list-field type-place place-secondary hidden-xs'])
    plak_list = [i.text.strip() for i in plak][1:]
    #plak_list.insert(9,'')
    #plak_list.insert(10,'')


    name = soup.find_all('h4',class_='list-field type-fullname')
    name_list = [i.text.strip() for i in name]

    startnr = soup.find_all('div',class_='list-field type-field')
    startnr_list = [i.text.removeprefix('Startnr.') for i in startnr][1:]

    verein = soup.find_all('div',class_='split list-field type-field')
    verein_list = [i.text.removeprefix('Verein/Team') for i in verein][1:]

    ak = soup.find_all('div',class_='list-field type-age_class')
    ak_list = [i.text.removeprefix('AK') for i in ak][1:]

    time = soup.find_all('div',class_='list-field type-time')
    time_list = [i.text.removeprefix('Finish').strip('Brutto') for i in time]
    finish_list = time_list[2:][::2]
    brutto_list = time_list[2:][1::2]
    
    name_url = soup.select('h4>a')
    name_url_list = [i.get('href').split('=')[4] for i in name_url]


    df = pd.DataFrame({
    'Platz': platz_list,
    'Pl.AK': plak_list,
    'Name': name_list,
    'Startnr.': startnr_list,
    'Verein/Team' : verein_list,
    'AK': ak_list,
    'Finish' : finish_list,
    'Brutto' : brutto_list, 
    'Url_IDP': name_url_list,
    'Year' : '2018'
    })
    
    haspa_marathon_2018 = pd.concat([haspa_marathon_2018,df])

In [79]:
haspa_marathon_2018

,Platz,Pl.AK,Name,Startnr.,Verein/Team,AK,Finish,Brutto,Url_IDP,Year
0,1,1,"Deksisa, Solomon (ETH)",7,–,MH / WH 1989-2000 (Hauptklasse),02:06:34,02:06:34,00001715DBE17E00002EC0E4&lang,2018
1,2,2,"Abate, Tadu (ETH)",14,–,MH / WH 1989-2000 (Hauptklasse),02:06:54,02:06:54,00001715DBE17E00002EC0E8&lang,2018
2,3,3,"Abshero, Ayele (ETH)",4,–,MH / WH 1989-2000 (Hauptklasse),02:07:19,02:07:19,00001715DBE17E00002EC0E1&lang,2018
3,4,1,"Yego, Solomon (KEN)",9,–,M / W 30 1984-1988,02:07:37,02:07:37,00001715DBE17E00002EC0E5&lang,2018
4,5,4,"Kiprotich, Stephen (UGA)",3,–,MH / WH 1989-2000 (Hauptklasse),02:07:57,02:07:57,00001715DBE17E00002EC0E0&lang,2018
...,...,...,...,...,...,...,...,...,...,...
149,7650,1262,"Lopes, Francisco (GER)",14148,BSV,M / W 45 1969-1973,06:12:56,06:23:56,99999915DBE17C00002DEC05&lang,2018
150,7651,1026,"Koepke, Martin (GER)",7099,–,M / W 35 1979-1983,06:13:22,06:18:10,99999915DBE17400002B7B1B&lang,2018
151,7652,291,"Reinert, Jürgen (GER)",15463,ERGOsports,M / W 60 1954-1958,06:13:53,06:14:04,99999915DBE17C00002E6DE1&lang,2018
152,7653,36,"Lesiak, Jan (GER)",16184,–,M / W 70 1944-1948,06:13:57,06:21:29,99999915DBE17C00002E74BB&lang,2018


In [ ]:
#merge 2018 to 2023 dataframes
haspa_marathon_2018_2023 = pd.concat([haspa_marathon_2023,
                                      haspa_marathon_2022,
                                      haspa_marathon_2021,
                                      haspa_marathon_2019,
                                      haspa_marathon_2018],ignore_index = True)

In [102]:
#quality checking
haspa_marathon_2018_2023.iloc[495]

Platz                                      496
Pl.AK                                       49
Name               López Olmeda, Antonio (ESP)
Startnr.                                 15316
Verein/Team    Castillejo&Cáceres Running Club
AK                            M/W 45 1974-1978
Finish                                03:00:27
Brutto                                03:00:52
Url_IDP                    HCHSK2IQ371A39&lang
Year                                      2023
Name: 495, dtype: object

In [ ]:
#exporting haspa_marathon_2018_2023 to .csv
haspa_marathon_2018_2023.to_csv('haspa_marathon_2018_2023.csv',index=False)

In [ ]:
#read .csv files for 2015, 2016, 2017 manually downloaded from haspa results website
haspa_marathon_2017 = pd.read_csv('../webscraping/Download/haspa_marathon_2017.csv',encoding = 'unicode_escape')
haspa_marathon_2016 = pd.read_csv('../webscraping/Download/haspa_marathon_2016.csv',encoding = 'unicode_escape')
haspa_marathon_2015 = pd.read_csv('../webscraping/Download/haspa_marathon_2015.csv',encoding = 'unicode_escape')

In [ ]:
#add year field
haspa_marathon_2017['Year'] = '2017'
haspa_marathon_2016['Year'] = '2016'
haspa_marathon_2015['Year'] = '2015'

In [93]:
#merge 2015 - 2017 tables
haspa_marathon_2015_2017 = pd.concat([haspa_marathon_2017,
                                      haspa_marathon_2016,
                                      haspa_marathon_2015],ignore_index = True)

In [95]:
#export haspa_marathon_2015_2017 to .csv
haspa_marathon_2015_2017.to_csv('haspa_marathon_2015_2017.csv',index=False)